In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import sklearn
sklearn.__version__

'0.24.1'

In [3]:
df = pd.read_csv('../data/processed/processed.csv')

In [4]:
df.shape

(6159, 9)

In [5]:
df.sample(3)

,kms_driven,owner,location,mileage,power,price,brand,engine,age
4360,9550.0,first,other,37.0,19.0,95000.0,Royal Enfield,350.0,7
5191,6000.0,second,other,40.0,19.0,55000.0,Bajaj,220.0,7
1860,5000.0,first,mumbai,40.0,19.8,150000.0,Royal Enfield,350.0,2


In [6]:
X = df.drop('price',axis=1)
y = df['price']

In [7]:
X.head()

,kms_driven,owner,location,mileage,power,brand,engine,age
0,19718.0,first,bangalore,65.00,17.0,Bajaj,180.0,5
1,24561.0,third,bangalore,35.00,25.0,KTM,200.0,9
2,38998.0,first,bangalore,40.00,19.8,Royal Enfield,350.0,10
3,7500.0,first,other,55.47,15.0,Hero,NaN,1
4,25300.0,first,pune,25.00,42.9,KTM,390.0,4


In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.25,random_state=11)

In [9]:
y = y.apply(np.log1p)

## Category Encoding

In [10]:
X.head()

,kms_driven,owner,location,mileage,power,brand,engine,age
0,19718.0,first,bangalore,65.00,17.0,Bajaj,180.0,5
1,24561.0,third,bangalore,35.00,25.0,KTM,200.0,9
2,38998.0,first,bangalore,40.00,19.8,Royal Enfield,350.0,10
3,7500.0,first,other,55.47,15.0,Hero,NaN,1
4,25300.0,first,pune,25.00,42.9,KTM,390.0,4


## Impute missing values

In [11]:
from sklearn.impute import KNNImputer

num_cols = [ col for col in X.columns if X[col].dtypes != 'object' ]
print(num_cols)

['kms_driven', 'mileage', 'power', 'engine', 'age']


In [12]:
imputer = KNNImputer(n_neighbors=7,weights='distance')

imputer.fit(X_train[num_cols])

# X_train[num_cols] = imputer.transform(X_train[num_cols])
# X_test[num_cols] = imputer.transform(X_test[num_cols])

X_train.loc[:][num_cols] = imputer.fit_transform(X_train[num_cols])
X_test.loc[:][num_cols] = imputer.transform(X_test[num_cols])

In [13]:
print('Missing Values in Train set',X_train.isnull().sum())
print('Missing Values in Test set',X_test.isnull().sum())

Missing Values in Train set kms_driven    0
owner         0
location      0
mileage       0
power         0
brand         0
engine        0
age           0
dtype: int64
Missing Values in Test set kms_driven    0
owner         0
location      0
mileage       0
power         0
brand         0
engine        0
age           0
dtype: int64


In [14]:
X_train.head()

,kms_driven,owner,location,mileage,power,brand,engine,age
6036,8800.0,first,other,35.000000,19.8,Royal Enfield,350.0,7
899,11012.0,first,other,32.000000,27.2,Royal Enfield,500.0,8
5923,19000.0,first,bangalore,25.000000,42.9,KTM,390.0,5
5332,59000.0,fourth,pune,65.000000,17.0,Bajaj,180.0,1
3137,17545.0,first,other,43.194823,14.6,Suzuki,150.0,3


In [15]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,MinMaxScaler

category_transformer = ColumnTransformer([
    ("kms_driven_engine_min_max_scaler",MinMaxScaler(),[0,6,3,4]),
    ("owner_ordinal_enc",OrdinalEncoder(categories=[['fourth','third','second','first']],handle_unknown='ignore',dtype=np.int16),[1]),
    ("brand_location_ohe",OneHotEncoder(sparse=False,handle_unknown='error',drop='first',),[2,5]),
],remainder='passthrough')



## Model Building

In [16]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from sklearn import set_config

set_config(display='diagram')

In [17]:

def build_pipeline_with_estimator(estimator):
    return Pipeline([
    ('category_transformer',category_transformer),
    ('estimator',estimator),
])


In [18]:
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
from sklearn.model_selection import cross_val_score
# MAPE
def mape(targets, predictions):
    return np.mean(np.abs((targets - predictions)) / targets) * 100

# Adjusted R^2
def adj_r2(ind_vars, targets, predictions):
    r2 = r2_score(targets, predictions)
    n = ind_vars.shape[0]
    k = ind_vars.shape[1]
    return 1-((1-r2)*(n-1)/(n-k-1))

# Model performance check
def model_perf(model, inp, out,cross_val=True):

    y_pred = model.predict(inp)
    y_act = out.values

    cross_val_ = cross_val_score(model, inp,out,cv=10).mean() if cross_val else None
    

    return pd.DataFrame({
                "RMSE": np.sqrt(mean_squared_error(y_act, y_pred)),
                "MAE": mean_absolute_error(y_act, y_pred),
                "MAPE": mape(y_act, y_pred),
                "R^2": r2_score(y_act, y_pred),
                "Adjusted R^2": adj_r2(inp, y_act, y_pred),
                "Cross Val Score (Mean)": cross_val_ if cross_val else None
           }, index=[0])

## LinearRegression

In [19]:
linear_regressor = build_pipeline_with_estimator(LinearRegression())

linear_regressor.fit(X_train,y_train)

print('Linear Regression Train Performance.\n')
print(model_perf(linear_regressor,X_train,y_train,True))

print('Linear Regression Test Performance.\n')
print(model_perf(linear_regressor,X_test,y_test,True))

Linear Regression Train Performance.

           RMSE           MAE      MAPE       R^2  Adjusted R^2  \
0  33140.859072  18664.679895  33.69047  0.712748       0.71225   

   Cross Val Score (Mean)  
0                0.714266  
Linear Regression Test Performance.

           RMSE           MAE       MAPE       R^2  Adjusted R^2  \
0  49775.510079  19536.011985  32.646759  0.518577      0.516061   

   Cross Val Score (Mean)  
0                 0.63524  


## RandomForest

In [20]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

model = build_pipeline_with_estimator(RandomForestRegressor())

params = {
    'estimator__criterion': ['mse','mae'],
    'estimator__n_estimators': [100,110,120,130],
    'estimator__max_depth': [5,10,15,20,25,30],
    'estimator__min_samples_split': range(2,20),
    'estimator__max_features': ['auto','sqrt','log2'],
}

model = RandomizedSearchCV(model,params,cv=5)

model.fit(X_train,y_train)

print('RandomForest Train Performance.\n')
print(model_perf(model,X_train,y_train,False))

print('RandomForest Test Performance.\n')
print(model_perf(model,X_test,y_test,False))

RandomForest Train Performance.

           RMSE          MAE       MAPE       R^2  Adjusted R^2  \
0  19362.639488  9103.655057  13.679596  0.901946      0.901776   

  Cross Val Score (Mean)  
0                   None  
RandomForest Test Performance.

           RMSE           MAE       MAPE       R^2  Adjusted R^2  \
0  45907.869673  14736.632605  21.441457  0.590485      0.588345   

  Cross Val Score (Mean)  
0                   None  


## XGBoost

In [21]:
from xgboost import XGBRegressor

xgboost = build_pipeline_with_estimator(XGBRegressor())

xgboost.fit(X_train,y_train)

print('xgboost Train Performance.\n')
print(model_perf(xgboost,X_train,y_train))

print('xgboost Test Performance.\n')
print(model_perf(xgboost,X_test,y_test))

xgboost Train Performance.

           RMSE          MAE       MAPE       R^2  Adjusted R^2  \
0  10155.746733  6955.756953  10.852464  0.973025      0.972978   

   Cross Val Score (Mean)  
0                 0.68516  
xgboost Test Performance.

           RMSE           MAE       MAPE      R^2  Adjusted R^2  \
0  48046.594844  14866.930505  22.165483  0.55144      0.549096   

   Cross Val Score (Mean)  
0                0.270941  


## KNeighborsRegressor

In [22]:
from sklearn.neighbors import KNeighborsRegressor

model = build_pipeline_with_estimator(KNeighborsRegressor())

params = {
    'estimator__n_neighbors': [3,5,7,9],
    'estimator__algorithm': ['ball_tree','kd_tree','brute'],
    'estimator__leaf_size': [5,10,30,32,35]
}

model = GridSearchCV(model,params,cv=5)

model.fit(X_train,y_train)

print('KNeighborsRegressor Train Performance.\n')
print(model_perf(model,X_train,y_train))

print('KNeighborsRegressor Test Performance.\n')
print(model_perf(model,X_test,y_test))

KNeighborsRegressor Train Performance.

           RMSE           MAE       MAPE       R^2  Adjusted R^2  \
0  32646.204942  14581.674226  20.626183  0.721259      0.720776   

   Cross Val Score (Mean)  
0                0.630024  
KNeighborsRegressor Test Performance.

           RMSE           MAE       MAPE      R^2  Adjusted R^2  \
0  50460.780833  17791.138873  24.092641  0.50523      0.502645   

   Cross Val Score (Mean)  
0                0.466425  


## GradientBoostingRegressor

In [23]:
from sklearn.ensemble import GradientBoostingRegressor

model = build_pipeline_with_estimator(GradientBoostingRegressor())

# params = {
#     'estimator__loss': ['ls','lad','huber','quantile'],
#     'estimator__learning_rate': [0.1,0.01,0.001],
#     'estimator__n_estimators': [100],
#     'estimator__criterion': ['friedman_mse','mse'],
# }

# model = RandomizedSearchCV(model,params,cv=5)

model.fit(X_train,y_train)

print('GradientBoostingRegressor Train Performance.\n')
print(model_perf(model,X_train,y_train))

print('GradientBoostingRegressor Test Performance.\n')
print(model_perf(model,X_test,y_test))

GradientBoostingRegressor Train Performance.

           RMSE           MAE       MAPE       R^2  Adjusted R^2  \
0  22398.137291  12574.535647  19.692266  0.868792      0.868565   

   Cross Val Score (Mean)  
0                0.751714  
GradientBoostingRegressor Test Performance.

           RMSE           MAE       MAPE       R^2  Adjusted R^2  \
0  48744.029948  15381.430692  23.369421  0.538323       0.53591   

   Cross Val Score (Mean)  
0                0.687179  
